In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import random
import json
import gzip
import csv
from textblob import TextBlob


In [5]:
dc_df = pd.read_csv('output.csv')

,user_id,name,time,rating,text,pics,resp,gmap_id
0,107074390665313624214,Monica Rivera,1558272538424,4.0,The venue is underground. Nice spot to grab a ...,[{'url': ['https://lh5.googleusercontent.com/p...,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
1,104256407771930872120,Petre Ene,1551027526726,4.0,"the food is really good, I had a great experie...",[{'url': ['https://lh5.googleusercontent.com/p...,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
2,114405225879020100938,Jacqui London,1566209658391,5.0,This yummy and inexpensive canteen in the bowe...,NaN,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
3,107718711786444128636,George W Jones 111,1575038843630,1.0,If it was possible to give less than 1 star I ...,NaN,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
4,115862049965497934152,Guillaume Lieb,1502129103994,4.0,Chocolates chips scones are delicious ! 👌,[{'url': ['https://lh5.googleusercontent.com/p...,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
5,103761185124557732666,Sungil Jung,1582396419875,3.0,Has good food pizza taste so so. The pizza is ...,NaN,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
6,113343329797428750284,Susan Hillberg,1581606117202,2.0,I am embarrassed as a DC native to have this c...,NaN,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
7,113228848273799766586,MeltingCORE2,1502581700374,2.0,I went there with my family today and was quit...,NaN,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
8,113455303404905301655,Travis H,1503428646289,4.0,Food has gotten much better here than it was l...,NaN,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
9,108642560086289718425,Keshava Mysore,1510920735052,4.0,High prices for not so big portions! But I thi...,NaN,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27


In [9]:
dc_df.head(20)

,user_id,name,time,rating,text,pics,resp,gmap_id
0,107074390665313624214,Monica Rivera,1558272538424,4.0,The venue is underground. Nice spot to grab a ...,[{'url': ['https://lh5.googleusercontent.com/p...,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
1,104256407771930872120,Petre Ene,1551027526726,4.0,"the food is really good, I had a great experie...",[{'url': ['https://lh5.googleusercontent.com/p...,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
2,114405225879020100938,Jacqui London,1566209658391,5.0,This yummy and inexpensive canteen in the bowe...,NaN,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
3,107718711786444128636,George W Jones 111,1575038843630,1.0,If it was possible to give less than 1 star I ...,NaN,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
4,115862049965497934152,Guillaume Lieb,1502129103994,4.0,Chocolates chips scones are delicious ! 👌,[{'url': ['https://lh5.googleusercontent.com/p...,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
5,103761185124557732666,Sungil Jung,1582396419875,3.0,Has good food pizza taste so so. The pizza is ...,NaN,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
6,113343329797428750284,Susan Hillberg,1581606117202,2.0,I am embarrassed as a DC native to have this c...,NaN,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
7,113228848273799766586,MeltingCORE2,1502581700374,2.0,I went there with my family today and was quit...,NaN,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
8,113455303404905301655,Travis H,1503428646289,4.0,Food has gotten much better here than it was l...,NaN,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
9,108642560086289718425,Keshava Mysore,1510920735052,4.0,High prices for not so big portions! But I thi...,NaN,NaN,0x89b7b7851b06ef6b:0x5f356b1eb1da27
